In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [5]:
# dataframe has 2 columns: 'Name' & 'Symbol'
df = pd.read_csv('SP50List.csv',index_col=0)
print(df.shape)
df.head()

(50, 2)


,Name,Symbol
0,Apple Inc.,AAPL
1,Microsoft Corp.,MSFT
2,Amazon.com Inc.,AMZN
3,NVIDIA Corp.,NVDA
4,Alphabet Inc. Class A,GOOGL


In [9]:
company_name = df['Name']
company_symbol = df['Symbol']
print(company_name[0],"& its symbol is ", company_symbol[0])

Apple Inc. & its symbol is  AAPL


In [10]:
url = "https://www.responsibilityreports.com"
page = requests.get(url).text
soup = BeautifulSoup(page, 'html.parser')
forms = soup.find_all('form')
print(forms)

[<form action="/Companies" method="get">
<input name="search" placeholder="Company or Ticker Symbol" type="text"/>
<input type="submit" value="Search"/>
</form>, <form action="/Companies">
<input name="search" placeholder="Company or Ticker Symbol" type="text"/>
<input type="submit" value="Search"/>
</form>, <form action="/Companies" method="get">
<span class="heading">Search for a Report</span>
<input name="search" placeholder="Company Name or Ticker Symbol" type="text"/>
<input type="submit" value="Search"/>
</form>]


In [11]:
payload = {'search': company_symbol[0]}
r = requests.post('https://www.responsibilityreports.com/Companies', data=payload)
print(r.status_code) # if 200, request is successful.

200


In [12]:
# get company name
search_req = BeautifulSoup(r.text, 'html.parser')
field_name = search_req.select_one('.companyName a')['href']
field_name

'/Company/apple-inc'

In [15]:
# link associated to company is not with company name. Find respective url.
url_with_comanyName = url + field_name
req_withCompanyName = requests.post(url_with_comanyName)
req_withCompanyName

<Response [200]>

In [16]:
search_req2 = BeautifulSoup(req_withCompanyName.text, 'html.parser')
link_for_companySearch = search_req2.select_one('.view_btn a')['href']
link_for_companySearch

'/Click/2285'

In [17]:
report_url = url + link_for_companySearch
report_url

'https://www.responsibilityreports.com/Click/2285'

In [18]:
# confirm status code
r = requests.get(report_url)  
r

<Response [200]>

In [19]:
year = '2022'
file_name = year+'_'+company_symbol[0]
with open(file_name+'.pdf', 'wb') as f:
    f.write(r.content)